In [6]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy.integrate import solve_ivp
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal, norm
import multiprocessing as mp

In [3]:
# Constants
G = 1.0                  # Gravitational constant (arbitrary units)
NUM_PARTICLES = 100      # Number of particles
TIME_STEP = 0.01         # Time step for integration
NUM_STEPS = 100          # Number of steps for simulation
PARTICLE_MASS = 1.0      # Mass of each particle
SOFTENING = 0.1          # Softening factor for gravitational force
KERNEL_RADIUS = 0.1      # Smoothing radius for SPH kernel
R_GAS_CONSTANT = 1.0     # Ideal gas constant (arbitrary units)
TEMPERATURE = 1.0        # Temperature (constant for simplicity)
VISCOSITY = 0.01         # Viscosity coefficient for Navier-Stokes

# Initialize particle positions and velocities
positions = np.random.rand(NUM_PARTICLES, 3) * 2 - 1  # Random 3D cube centered at origin
velocities = np.zeros((NUM_PARTICLES, 3))             # Start with zero velocity
densities = np.zeros(NUM_PARTICLES)
pressures = np.zeros(NUM_PARTICLES)

In [7]:
def gaussian_kernel(r, h=KERNEL_RADIUS):
    """SPH Gaussian kernel function."""
    return norm.pdf(r,scale=h)/h

def compute_densities():
    """Compute density for each particle based on neighboring particles using SPH Gaussian kernel."""
    global densities
    # Pairwise distance between particles
    distances = cdist(positions, positions)  # Shape: (N, N)
    kernel_values = gaussian_kernel(distances, KERNEL_RADIUS)
    np.fill_diagonal(kernel_values, 0)  # Ignore self-contribution
    densities = PARTICLE_MASS * np.sum(kernel_values, axis=1)

def compute_pressures():
    """Compute pressure for each particle using the ideal gas law."""
    global pressures
    pressures = densities * R_GAS_CONSTANT * TEMPERATURE  # Ideal gas law: P = rho * R * T

def compute_pressure_forces():
    """Compute pressure force on each particle using the SPH gradient."""
    pressure_forces = np.zeros((NUM_PARTICLES, 3))

    # Pairwise distance vectors and magnitudes
    r_vec = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]  # Shape: (N, N, 3)
    distances = np.linalg.norm(r_vec, axis=2) + SOFTENING
    np.fill_diagonal(distances, np.inf)  # Avoid self-interaction

    # Kernel gradient for each particle pair
    kernel_gradient = -gaussian_kernel(distances, KERNEL_RADIUS) / distances[..., np.newaxis] * r_vec
    np.fill_diagonal(kernel_gradient[:, :, 0], 0)  # Zero out self-contribution for x component
    np.fill_diagonal(kernel_gradient[:, :, 1], 0)  # Zero out self-contribution for y component
    np.fill_diagonal(kernel_gradient[:, :, 2], 0)  # Zero out self-contribution for z component

    # Calculate pressure force contributions
    for i in range(NUM_PARTICLES):
        pressure_contrib = PARTICLE_MASS * (pressures[i] / densities[i]*2 + pressures / densities*2)
        pressure_forces[i] = np.sum(pressure_contrib[:, np.newaxis] * kernel_gradient[i], axis=0)

    return pressure_forces

def compute_viscous_forces():
    """Compute viscous forces based on velocity differences between particles (simplified Navier-Stokes)."""
    viscous_forces = np.zeros((NUM_PARTICLES, 3))
    r_vec = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]  # Shape: (N, N, 3)
    distances = np.linalg.norm(r_vec, axis=2) + SOFTENING
    np.fill_diagonal(distances, np.inf)

    # Relative velocities between particles
    rel_velocities = velocities[:, np.newaxis, :] - velocities[np.newaxis, :, :]
    viscous_term = VISCOSITY * (rel_velocities / (distances[..., np.newaxis]**2))
    viscous_forces = np.sum(viscous_term, axis=1) * PARTICLE_MASS

    return viscous_forces

def compute_gravitational_forces():
    """Compute gravitational forces between particles using vectorized calculations."""
    r_vec = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]  # Shape: (N, N, 3)
    distances = np.linalg.norm(r_vec, axis=2) + SOFTENING
    np.fill_diagonal(distances, np.inf)

    # Gravitational force calculation
    grav_forces = G * PARTICLE_MASS*2 * r_vec / distances[:, :, np.newaxis]*3
    net_grav_forces = np.sum(grav_forces, axis=1)

    return net_grav_forces

def compute_total_forces():
    """Compute total forces on each particle (pressure, viscosity, and gravity)."""
    with mp.Pool(3) as pool:
        pressure_forces = pool.apply_async(compute_pressure_forces).get()
        viscous_forces = pool.apply_async(compute_viscous_forces).get()
        gravitational_forces = pool.apply_async(compute_gravitational_forces).get()

    total_forces = pressure_forces + viscous_forces + gravitational_forces
    return total_forces

def derivatives(t, y):
    """Function to compute derivatives for solve_ivp.

    y contains positions and velocities concatenated:
    y = [x1, y1, z1, vx1, vy1, vz1, x2, y2, z2, vx2, vy2, vz2, ...]
    """
    # Reshape y into positions and velocities
    global positions, velocities
    pos = y[:NUM_PARTICLES * 3].reshape((NUM_PARTICLES, 3))
    vel = y[NUM_PARTICLES * 3:].reshape((NUM_PARTICLES, 3))

    positions, velocities = pos, vel  # Update global positions and velocities for density calculation

    # Calculate density, pressure, and forces
    compute_densities()
    compute_pressures()
    accelerations = compute_total_forces() / PARTICLE_MASS

    dydt = np.concatenate([vel.flatten(), accelerations.flatten()])
    return dydt



In [8]:
# Initial conditions for solve_ivp
initial_conditions = np.concatenate([positions.flatten(), velocities.flatten()])

# Run the solver over time
solution = solve_ivp(derivatives, [0, NUM_STEPS * TIME_STEP], initial_conditions, method='RK45', t_eval=np.linspace(0, NUM_STEPS * TIME_STEP, NUM_STEPS))

# Retrieve positions over time for visualization
positions_over_time = solution.y[:NUM_PARTICLES * 3].reshape((NUM_PARTICLES, 3, -1))



ValueError: operands could not be broadcast together with shapes (100,100,100) (100,100,3) 

In [ ]:
# Visualization setup for 3D rendering in Google Colab
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
frames = []

def update(frame):
    ax.clear()
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
    ax.scatter(positions_over_time[:, 0, frame], positions_over_time[:, 1, frame], positions_over_time[:, 2, frame], s=5, c='blue')

ani = FuncAnimation(fig, update, frames=NUM_STEPS, interval=50)
writer = PillowWriter(fps=20)
ani.save("sph_simulation.gif", writer=writer)

# Displaying the GIF (for Google Colab)
from IPython.display import Image
Image(open("sph_simulation.gif",'rb').read())